In [66]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]=os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")


In [67]:
#### data injection

from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://www.javatpoint.com/machine-learning")
loader

In [68]:
doc=loader.load()

In [69]:
doc

[Document(metadata={'source': 'https://www.javatpoint.com/machine-learning', 'title': 'Machine Learning Tutorial', 'description': ' Machine Learning: What It is, Tutorial, Definition, Types with Machine Learning, Machine Learning Tutorial, Machine Learning Introduction, What is Machine Learning, Data Machine Learning, Applications of Machine Learning, Machine Learning vs Artificial Intelligence, dimensionality reduction, deep learning, etc.', 'language': 'en'}, page_content='Machine Learning TutorialTutorials×PythonPython Django Numpy Pandas Tkinter Pytorch Flask OpenCVAI, ML and Data ScienceArtificial Intelligence Machine Learning Data Science Deep Learning TensorFlow Artificial Neural Network Matplotlib Python ScipyJavaJava Servlet JSP Spring Boot Spring Framework Hibernate JavaFX Java Web ServicesB.Tech and MCADBMS Data Structures Operating System Computer Network DAA Computer Organization Software Engineering Data MiningWeb TechnologyHTML CSS JavaScript Jquery Angular-8 React JS Re

In [84]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)

In [85]:
chunk=splitter.split_documents(doc)
chunk

[Document(metadata={'source': 'https://www.javatpoint.com/machine-learning', 'title': 'Machine Learning Tutorial', 'description': ' Machine Learning: What It is, Tutorial, Definition, Types with Machine Learning, Machine Learning Tutorial, Machine Learning Introduction, What is Machine Learning, Data Machine Learning, Applications of Machine Learning, Machine Learning vs Artificial Intelligence, dimensionality reduction, deep learning, etc.', 'language': 'en'}, page_content='Machine Learning TutorialTutorials×PythonPython Django Numpy Pandas Tkinter Pytorch Flask OpenCVAI, ML and Data ScienceArtificial Intelligence Machine Learning Data Science Deep Learning TensorFlow Artificial Neural Network Matplotlib Python ScipyJavaJava Servlet JSP Spring Boot Spring Framework Hibernate JavaFX Java Web ServicesB.Tech and MCADBMS Data Structures Operating System Computer Network DAA Computer Organization Software Engineering Data MiningWeb TechnologyHTML CSS JavaScript Jquery'),
 Document(metadata

In [86]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()

In [87]:
from langchain_community.vectorstores import FAISS
vector_db=FAISS.from_documents(chunk,embedding)

In [88]:
vector_db

In [89]:
vector_db.save_local("faiss_index")

In [92]:
query="What is machine learning"
result=vector_db.similarity_search(query)

In [93]:
result[0].page_content

'our machine learning tutorial.A rapidly developing field of technology, machine learning allows computers to automatically learn from previous data. For building mathematical models and making predictions based on historical data or information, machine learning employs a variety of algorithms. It is currently being used for a variety of tasks, including speech recognition, email filtering, auto-tagging on Facebook, a recommender system, and image recognition.You will learn about the many'

In [94]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [95]:
### Retrival chain , Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """ Answer the following question based on the provided context :
    <context>{context}</context>
    """
)
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Answer the following question based on the provided context :\n    <context>{context}</context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f9548f49960>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f9548e15600>, root_client=<openai.OpenAI object at 0x7f954932fc40>, root_async_client=<openai.AsyncOpenAI object at 0x7f9548f4a5f0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, conf

In [101]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"what is machine learning?",
    "context" : [Document(page_content="In the real world, we are surrounded by humans who can learn everything from their experiences with their learning capability, and we have computers or machines which work on our instructions. But can a machine also learn from experiences or past data like a human does? So here comes the role of Machine Learning.")]
})

'Yes, a machine can learn from experiences or past data similar to how humans do, and this is achieved through Machine Learning. Machine Learning is a field of artificial intelligence that enables computers or machines to learn from data and improve their performance over time without being explicitly programmed. It involves the use of algorithms and statistical models to analyze and interpret patterns in data, allowing machines to make predictions or decisions based on that data.'

In [103]:
### Retriever
retriever=vector_db.as_retriever()

In [104]:
from langchain.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriever,document_chain)

In [105]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f95494632b0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Answer the following question based on the provided context :\n    <context>{context}</context>\n    '), additional_kwargs={})])
            | ChatOpenA

In [106]:
## response from the openai
response=retriever_chain.invoke({"input":"what is machine learning?"})

In [107]:
response

{'input': 'what is machine learning?',
 'context': [Document(metadata={'source': 'https://www.javatpoint.com/machine-learning', 'title': 'Machine Learning Tutorial', 'description': ' Machine Learning: What It is, Tutorial, Definition, Types with Machine Learning, Machine Learning Tutorial, Machine Learning Introduction, What is Machine Learning, Data Machine Learning, Applications of Machine Learning, Machine Learning vs Artificial Intelligence, dimensionality reduction, deep learning, etc.', 'language': 'en'}, page_content='our machine learning tutorial.A rapidly developing field of technology, machine learning allows computers to automatically learn from previous data. For building mathematical models and making predictions based on historical data or information, machine learning employs a variety of algorithms. It is currently being used for a variety of tasks, including speech recognition, email filtering, auto-tagging on Facebook, a recommender system, and image recognition.You w

In [108]:
response["answer"]

'What is machine learning, and why is it important?\n\nMachine learning is a subset of artificial intelligence focused on creating algorithms that enable computers to learn from data and past experiences without being explicitly programmed. It is important because it allows computers to perform complex tasks that are difficult for humans to implement directly. By analyzing vast amounts of data, machine learning systems can build models and make predictions, thereby simplifying tasks such as speech recognition, email filtering, and recommender systems. This capability is essential because humans are limited in their ability to manually process large data sets, whereas machine learning can automate and enhance these processes.'